In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.prompts import ChatPromptTemplate


In [5]:
policy_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "you are an HR policy writer creating policies"
     "you will recieve an HR POLICY Template which is extracted from a DOCX file.\n"
     "Generate a New policy that follows same structure/headings style, but write ORIGINAL content.\n"
     "Rules:\n"
     "- Do not copy long phrases verbatiam from the template"
     "- Generate a concise and Well-structure policy"),
     ("user", 
      "TEMPLATE (reference): \n---\n{template_text}"
      "Generate policy:\n"
      "- company: {company_name}\n"
      "- company size: {company_size}\n"
      "- tone: {tone} \n"
      "- company category: {company_category}\n"
      "- country context: {country}\n"
      "Return only Markdown")

])

In [6]:
loader = Docx2txtLoader("../docs/Company Policies/Attendance Policy.docx")
docs = loader.load()
template_text = "\n\n".join(doc.page_content for doc in docs)

In [7]:
print(template_text)

Attendance Policy



1. OVERVIEW



Each employee at [Company Name] is responsible for punctual and consistent attendance. Employees should arrive on time, be prepared to work, and on schedule. Employees are also expected to stay at work for the whole of their shift. It is inconvenient to arrive late, leave early, or miss other scheduled hours, and it must be avoided.



This policy does not apply to FMLA-covered (FMLA - Family and Medical Leave Act) absences or leave taken as a reason

able accommodation under the Americans with Disabilities Act (ADA). They have their own policies that cover these exceptions.



2. OBJECTIVE

The goal of this policy is to lay out [Company Names] policies and processes for dealing with employee absences and tardiness in order to increase the company's efficiency and reduce unscheduled absences.



3. ATTENDANCE INFRACTIONS CALCULATION



Absent with calls - 1 Point 

Absent with no calls - 2 Points 

Tardy - ½ Point

Early Departure - ½ Point

Returnin

In [8]:
from langchain_google_vertexai import ChatVertexAI
llm = ChatVertexAI(
    model_name="gemini-2.5-flash-lite",
    temperature=0.2,
    max_output_tokens=2048,
)

C:\Users\prath\AppData\Local\Temp\ipykernel_12752\1003980041.py:2: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(
C:\Users\prath\AppData\Local\Temp\ipykernel_12752\1003980041.py:2: LangChainDeprecationWarning: The class `ChatVertexAI` was deprecated in LangChain 3.2.0 and will be removed in 4.0.0. An updated version of the class exists in the `langchain-google-genai package and should be used instead. To use it run `pip install -U `langchain-google-genai` and import as `from `langchain_google_genai import ChatGoogleGenerativeAI``.
  llm = ChatVertexAI(


In [9]:
chain = policy_prompt | llm
response = chain.invoke(
    {
        "template_text": template_text,
        "company_name": "Acme Corp",
        "company_size": "10000 employees",
        "tone": "professional",
        "company_category": "Information Technology",
        "country": "India"
    })

c:\temp\agentic\RAG\Projects\hr-helpdesk\experiments\.venv\Lib\site-packages\langsmith\client.py:498: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18,id=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18; trace=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18,id=019c5f41-a928-75c2-8731-f564f129babb; trace=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18,id=019c5f41-a944-78b3-9c8c-b69212f6673f


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18,id=019c5f41-a944-78b3-9c8c-b69212f6673f; trace=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18,id=019c5f41-a8ec-7dd2-84d8-5f505e7a3f18


In [10]:
response.pretty_print()

================================== Ai Message ==================================

```markdown
# Employee Punctuality and Absence Policy

## 1. Introduction

Acme Corp values the dedication and commitment of its employees. Punctual and consistent attendance is crucial for maintaining operational efficiency and ensuring seamless collaboration within our Information Technology environment. This policy outlines the expectations for employee attendance, reporting procedures for absences, and the consequences of non-compliance.

This policy does not supersede or alter provisions related to legally protected leaves such as those under the Maternity Benefit Act, the Employees' State Insurance Act, or any other applicable Indian labor laws, nor does it affect accommodations provided under disability-related legislation.

## 2. Purpose

The objective of this policy is to establish clear guidelines and procedures for managing employee attendance, tardiness, and unscheduled absences. By adhering t

In [11]:
import os

def combine_dir_with_markdown(dir_path, docx_path):
    # Extract filename from docx path
    filename = os.path.basename(docx_path)
    
    # Remove extension and convert to markdown name
    name_without_ext = os.path.splitext(filename)[0]
    markdown_name = name_without_ext.replace(" ", "_") + ".md"
    
    # Combine directory path with markdown filename
    return os.path.join(dir_path, markdown_name)


In [ ]:
from langchain_community.document_loaders import DirectoryLoader
chain = policy_prompt | llm

directory_loader = DirectoryLoader(
    "../docs/Company Policies",
    glob="*.docx",
    loader_cls=Docx2txtLoader)

docs = directory_loader.load()
for doc in docs:
    template_text = "\n\n".join(doc.page_content for doc in docs)
    path = combine_dir_with_markdown(
        "../generated_data", doc.metadata['source'])
    response = chain.invoke(
        {
            "template_text": template_text,
            "company_name": "Acme Corp",
            "company_size": "10000 employees",
            "tone": "professional",
            "company_category": "Information Technology",
            "country": "India"
        })
    with open(path, 'w') as f:
        f.write(response.content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019c5f47-8c29-7fd2-9a60-7c80ffa5d7c9,id=019c5f47-8c29-7fd2-9a60-7c80ffa5d7c9; trace=019c5f47-8c29-7fd2-9a60-7c80ffa5d7c9,id=019c5f47-8c2e-7c50-a264-6759e533bce0; trace=019c5f47-8c29-7fd2-9a60-7c80ffa5d7c9,id=019c5f47-8c2e-7c50-a264-6759e533bce0; trace=019c5f47-8c29-7fd2-9a60-7c80ffa5d7c9,id=019c5f47-8c31-7f42-ad2d-207f3499f164
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019c5f47-8c29-7fd2-9a60-7c80ffa5d7c9,id=019c5f47-8c31-7f42-ad2d-207f3499f164; trace=019c5f47-

KeyboardInterrupt: 

Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)))"))
Content-Length: 2954780
API Key: trace=019c5f4e-e6fc-7220-b0a0-9586fd1a6c71,id=019c5f4e-e70a-7870-b534-104d24849193; trace=019c5f4e-e6fc-7220-b0a0-9586fd1a6c71,id=019c5f4e-e6fc-7220-b0a0-9586fd1a6c71; trace=019c5f4e-fd9a-7d23-8d32-400be8359e9f,id=019c5f4e-fd9a-7d23-8d32-400be8359e9f; trace=019c5f4e-fd9a-7d23-8d32-400be8359e9f,id=019c5f4e-fd9c-79a1-93b4-47aab49c0733; trace=019c5f4e-fd9a-7d23-8d32-400be8359e9f,id=019c5f4e-fd9c-79a1-93b4-47aab49c0733; trace=019c5f4e-fd9a-7d23-8d32-400be8